# Language Model Training

In [4]:
# Clone the repository
!git clone https://github.com/inner-LMNt/Self-GPT.git
%cd Self-GPT

# Install requirements
!pip install -r requirements.txt

!pwd
!ls

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [9]:
import torch
import json
import time
import os
import sys

sys.path.append(os.getcwd())

from models.LLM.config import Config
from models.LLM.GPT import GPT
from models.LLM.bigram import BigramLanguageModel, BigramNoAttention
from models.LLM.trigram import TrigramLanguageModel, TrigramNoAttention

In [11]:
def load_json(file):
    with open(file, 'r') as f:
        data = json.load(f)
    return data

def n_gram(number=2):
    config = Config()
    # torch.manual_seed(config.seed)

    model_name = "shakespeare_model"

    if number == 2:
        path = config.checkpoint_dir + f"/bigram/{model_name}.pth"
        LLM = BigramLanguageModel().to(config.device)
    else:
        path = config.checkpoint_dir + f"/trigram/{model_name}.pth"
        LLM = TrigramLanguageModel().to(config.device)

    try:
        LLM.load_state_dict(torch.load(path, map_location=config.device))
        print("Model loaded successfully:", path)
        print("Training...")
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Training model from scratch...")

    data = load_json(config.data_dir + '/' + config.data_file)
    data = torch.tensor(data, dtype=torch.long)

    split = int(0.9 * len(data))
    train_data, val_data = data[:split], data[split:]

    def generate_batch(train):
        data = train_data if train else val_data
        offsets = torch.randint(len(data) - config.context_len, (config.batch_size,))
        inputs = torch.stack([data[i:i+config.context_len] for i in offsets])
        targets = torch.stack([data[i+1:i+config.context_len+1] for i in offsets])
        return inputs.to(config.device), targets.to(config.device)

    start = time.time()
    optimizer = torch.optim.Adam(LLM.parameters(), lr=config.learning_rate)
    for _ in range(10000):
        x, y = generate_batch(True)
        logits, loss = LLM.forward(x, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward() # Gradients
        optimizer.step() # Update weights

    print("\n______________________________________________________")
    print(f"Training loss: {loss.item()}, Validation loss: {LLM.forward(*generate_batch(False))[1].item()}")
    print(f"Training time: {time.time() - start} seconds")
    print("______________________________________________________\n")

    print(f"Saving model to {path}")
    try:
        torch.save(LLM.state_dict(), path)
        print("Model saved successfully")
    except Exception as e:
        print(f"Error saving model: {e}")

## Run Training

In [12]:
n_gram(2) # or 3
print("\nFinished.")

Error loading model: [Errno 2] No such file or directory: './models/checkpoints/bigram/shakespeare_model.pth'
Training model from scratch...

______________________________________________________
Training loss: 2.3169105052948, Validation loss: 2.2330212593078613
Training time: 96.30599761009216 seconds
______________________________________________________

Saving model to ./models/checkpoints/bigram/shakespeare_model.pth
Model saved successfully

Finished.


## Download Trained Model

After training, you can download the model using the code or just right-
clicking the file.

In [13]:
from google.colab import files

config = Config()
model_type = "bigram"
model_name = "shakespeare_model"

# Adjust the path based on which model you trained
path = config.checkpoint_dir + f"/{model_type}/{model_name}.pth"

if os.path.exists(path):
    files.download(path)
else:
    print(f"Model file not found at {path}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>